# Program: Projeto_NER_v05.ipynb (python3) 
## Author: Laércio Lucchesi
### Date: 17.07.2021

In [1]:
#importação de bibliotecas 
import numpy as np
from difflib import SequenceMatcher
import pandas as pd
import spacy
import re
from tqdm import tqdm
import operator
from operator import itemgetter
import time
from sklearn.model_selection import train_test_split
from spacy.tokens import DocBin
import collections

In [5]:
#importa arquivo das descricoes normalizadas enriquecidas manualmente
descricoes=pd.read_csv("desc_norm_enriquecida.csv",sep=",")

#arruma a descricao normalizada retirando brancos e caracteres especiais desnecessários
for i in range (0,len(descricoes['descricao_normalizada'])):
    descricoes.at[i,'descricao_normalizada']=" ".join(re.sub(r'[^\w\s]',' '," ".join(descricoes.at[i,'descricao_normalizada'].split())).split())
    

In [ ]:
#importa arquivo da base de arroz limpa (sem produtos não pertencentes a classe de arroz)
base_limpa=pd.read_csv("base_arroz_limpa.csv",sep=",")

#arruma as descricoes retirando caracteres especiais e brancos desnecessários
for i in range (0,len(base_limpa['descricao_estabelecimento'])):
    base_limpa.at[i,'descricao_estabelecimento']=" ".join(re.sub(r'[^\w\s]',' '," ".join(base_limpa.at[i,'descricao_estabelecimento'].split())).split())
    base_limpa.at[i,'descricao_normalizada']=" ".join(re.sub(r'[^\w\s]',' '," ".join(base_limpa.at[i,'descricao_normalizada'].split())).split())

In [ ]:
# --------------------------------------------------------------------------------------
#  programa para enriquecer a base de dados
# --------------------------------------------------------------------------------------
#  a partir das bases de dados de arroz e das descrições normalizadas enriquecidas é 
#  gerada uma base de dados de arroz enriquecida.
# --------------------------------------------------------------------------------------
#  Inputs: desc_norm_enriquecida.csv, base_arroz_limpa.csv
#  Output: base_arroz_enriquecida.csv
# --------------------------------------------------------------------------------------

#cria o dataframe df_base_arroz_enriquecida
nomes_colunas=['ean','descricao','start_marca','end_marca','label_marca','start_marcaX','end_marcaX','label_marcaX','start_peso','end_peso','label_peso','start_tipo','end_tipo','label_tipo','start_prod','end_prod','label_prod','start_prodX','end_prodX','label_prodX']
df_base_arroz_enriquecida = pd.DataFrame(columns=nomes_colunas)

arroz_enriq_list = []

for i in tqdm(range(len(base_limpa))):

    #ean
    ean=base_limpa.iloc[i]['ean']
    
    #descricao do estabelecimento
    descricao=base_limpa.iloc[i]['descricao_estabelecimento']
    
    analise=[]

    for atributo in ['PROD', 'PROD_X', 'TIPO', 'MARCA', 'MARCA_X','PESO','EMBAL']:
    
        #busca o atributo baseado na descricao normalizada e armazena em palavra
        #ean seria um candidato para chave mas existem casos de mais de uma ean para o mesmo produto
        #entao optei pela descricao_normalizada como chave
        palavra=descricoes[descricoes['descricao_normalizada'] == base_limpa.iloc[i]['descricao_normalizada']].iloc[0][atributo]
        
        #analise de similaridade
        analise_similaridade=mais_similar(palavra, descricao)
        palavra_mais_similar=analise_similaridade[0]
        grau_similaridade=analise_similaridade[1]
        pos_inicio=analise_similaridade[2]
        pos_fim=analise_similaridade[3]

        analise.append([atributo,palavra_mais_similar,grau_similaridade,pos_inicio,pos_fim])
    
    #elimina eventuais conflitos (mesma palavra para mais de um atributo)
    analise=elimina_conflitos(analise)
    
    #converte analise numa tupla de tuplas (para melhor performance)
    analiset = tuple(tuple(sub) for sub in analise)

    #cria uma linha (formato dicionário) da base de arroz enriquecida a partir da tupla analiset
    linha={
    'ean': ean,
    'descricao':descricao,
    'start_marca':analiset[3][3],
    'end_marca':analiset[3][4],
    'label_marca':'MARCA',
    'start_marcaX':analiset[4][3],
    'end_marcaX':analiset[4][4],
    'label_marcaX':'MARCA_X',
    'start_peso':analiset[5][3],
    'end_peso':analiset[5][4],
    'label_peso':'PESO',
    'start_tipo':analiset[2][3],
    'end_tipo':analiset[2][4],
    'label_tipo':'TIPO',
    'start_prod':analiset[0][3],
    'end_prod':analiset[0][4],
    'label_prod':'PROD',
    'start_prodX':analiset[1][3],
    'end_prodX':analiset[1][4],
    'label_prodX':'PROD_X',
    'start_embal':analiset[6][3],
    'end_embal':analiset[6][4],
    'label_embal':'EMBAL'        
    }
    
    #adiciona a linha na lista
    arroz_enriq_list.append(linha)

#atualiza o dataframe da base de arroz enriquecida
df_base_arroz_enriquecida = pd.DataFrame.from_dict(arroz_enriq_list)
    
#salva dataframe num csv
df_base_arroz_enriquecida.to_csv('base_arroz_enriquecida.csv',index=False)


In [ ]:
#importa arquivo da base de arroz enriquecida
df_arroz_total=pd.read_csv("base_arroz_enriquecida.csv",sep=",")

In [ ]:
# separa um dataframe para treino e outro para teste

df_arroz_treino, df_arroz_teste = train_test_split(df_arroz_total, test_size=0.2)

#reseta o índice
df_arroz_treino = df_arroz_treino.reset_index()
df_arroz_teste = df_arroz_teste.reset_index()
        
#salva os dataframes em formato csv
df_arroz_treino.to_csv('arroz_treino.csv',index=False)
df_arroz_teste.to_csv('arroz_teste.csv',index=False)

In [ ]:
#recupera bases de treino e teste dos arquivos csv
df_arroz_treino=pd.read_csv("arroz_treino.csv",sep=",")
df_arroz_teste=pd.read_csv("arroz_teste.csv",sep=",")

In [ ]:
#constroi arquivo json com dados de treino
c1=[]
c2=[]

for i in tqdm(range(0,len(df_arroz_treino['descricao']))):
    c1.append(df_arroz_treino['descricao'][i])
    linha=[]
    if not (df_arroz_treino['start_marca'][i]==0 and df_arroz_treino['end_marca'][i]==0):
        linha=linha+[(df_arroz_treino['start_marca'][i],df_arroz_treino['end_marca'][i],df_arroz_treino['label_marca'][i])]
    if not (df_arroz_treino['start_marcaX'][i]==0 and df_arroz_treino['end_marcaX'][i]==0):
        linha=linha+[(df_arroz_treino['start_marcaX'][i],df_arroz_treino['end_marcaX'][i],df_arroz_treino['label_marcaX'][i])]
    if not (df_arroz_treino['start_peso'][i]==0 and df_arroz_treino['end_peso'][i]==0):
        linha=linha+[(df_arroz_treino['start_peso'][i],df_arroz_treino['end_peso'][i],df_arroz_treino['label_peso'][i])]
    if not (df_arroz_treino['start_tipo'][i]==0 and df_arroz_treino['end_tipo'][i]==0):
        linha=linha+[(df_arroz_treino['start_tipo'][i],df_arroz_treino['end_tipo'][i],df_arroz_treino['label_tipo'][i])]
    if not (df_arroz_treino['start_embal'][i]==0 and df_arroz_treino['end_embal'][i]==0):
        linha=linha+[(df_arroz_treino['start_embal'][i],df_arroz_treino['end_embal'][i],df_arroz_treino['label_embal'][i])]
    if not (df_arroz_treino['start_prod'][i]==0 and df_arroz_treino['end_prod'][i]==0):
        linha=linha+[(df_arroz_treino['start_prod'][i],df_arroz_treino['end_prod'][i],df_arroz_treino['label_prod'][i])]
    if not (df_arroz_treino['start_prodX'][i]==0 and df_arroz_treino['end_prodX'][i]==0):
        linha=linha+[(df_arroz_treino['start_prodX'][i],df_arroz_treino['end_prodX'][i],df_arroz_treino['label_prodX'][i])]
      
    c2.append({"entities":linha})
   
TRAIN_DATA = list(zip(c1,c2))

In [ ]:
#constroe arquivo json com dados de teste
c1=[]
c2=[]

for i in tqdm(range(0,len(df_arroz_teste['descricao']))):
    c1.append(df_arroz_teste['descricao'][i])
    linha=[]
    if not (df_arroz_teste['start_marca'][i]==0 and df_arroz_teste['end_marca'][i]==0):
        linha=linha+[(df_arroz_teste['start_marca'][i],df_arroz_teste['end_marca'][i],df_arroz_teste['label_marca'][i])]
    if not (df_arroz_teste['start_marcaX'][i]==0 and df_arroz_teste['end_marcaX'][i]==0):
        linha=linha+[(df_arroz_teste['start_marcaX'][i],df_arroz_teste['end_marcaX'][i],df_arroz_teste['label_marcaX'][i])]
    if not (df_arroz_teste['start_peso'][i]==0 and df_arroz_teste['end_peso'][i]==0):
        linha=linha+[(df_arroz_teste['start_peso'][i],df_arroz_teste['end_peso'][i],df_arroz_teste['label_peso'][i])]
    if not (df_arroz_teste['start_tipo'][i]==0 and df_arroz_teste['end_tipo'][i]==0):
        linha=linha+[(df_arroz_teste['start_tipo'][i],df_arroz_teste['end_tipo'][i],df_arroz_teste['label_tipo'][i])]
    if not (df_arroz_teste['start_embal'][i]==0 and df_arroz_teste['end_embal'][i]==0):
        linha=linha+[(df_arroz_teste['start_embal'][i],df_arroz_teste['end_embal'][i],df_arroz_teste['label_embal'][i])]
    if not (df_arroz_teste['start_prod'][i]==0 and df_arroz_teste['end_prod'][i]==0):
        linha=linha+[(df_arroz_teste['start_prod'][i],df_arroz_teste['end_prod'][i],df_arroz_teste['label_prod'][i])]
    if not (df_arroz_teste['start_prodX'][i]==0 and df_arroz_teste['end_prodX'][i]==0):
        linha=linha+[(df_arroz_teste['start_prodX'][i],df_arroz_teste['end_prodX'][i],df_arroz_teste['label_prodX'][i])]
      
    c2.append({"entities":linha})
   
TEST_DATA = list(zip(c1,c2))

In [ ]:
#transforma TRAIN_DATA para formato Spacy 3.0 - DocBin
from spacy.tokens import DocBin

nlp = spacy.blank("pt") # load a new spacy model
db = DocBin() # create a DocBin object

for text, annot in tqdm(TRAIN_DATA): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
                span = doc.char_span(int(start), int(end), label=label, alignment_mode="contract")
                if span is not None:
                    ents.append(span)
    
    doc.ents = ents # label the text with the ents
    db.add(doc)

db.to_disk("./arroz_treino.spacy") # save the docbin object

In [ ]:
#transforma TEST_DATA para formato Spacy 3.0 - DocBin
from tqdm import tqdm
from spacy.tokens import DocBin

nlp = spacy.blank("pt") # load a new spacy model
db = DocBin() # create a DocBin object

for text, annot in tqdm(TEST_DATA): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
                span = doc.char_span(int(start), int(end), label=label, alignment_mode="contract")
                if span is not None:
                    ents.append(span)
                doc.ents = ents # label the text with the ents
    db.add(doc)
    
db.to_disk("./arroz_teste.spacy") # save the docbin object

In [ ]:
#--------------------------------------------------------------------
#treinamento e avaliação de um modelo usando a linha de comando
#--------------------------------------------------------------------

#------- configuração do treinamento --------
#python -m spacy init fill-config base_config.cfg config.cfg

#------- treinamento --------
#python -m spacy train config.cfg --output ./output --paths.train ./arroz_treino.spacy --paths.dev ./arroz_teste.spacy

#------- avaliação --------
#python -m spacy evaluate ./output/model-best arroz_teste.spacy

In [2]:
#testa algumas descrições

#carrega o melhor modelo treinado
nlp = spacy.load(R".\output\model-last") 

# alguns testes com uma marca nova
descricao_1 = nlp("ARR PARB T 1 NOVAM PCT 1K") 
descricao_2 = nlp("ARRO NMARCA PARBO TIPO 1 PREM 1 KG") 
descricao_3 = nlp("AR NOVAMARC TIPO 1 1KG PROMOCAO") 
descricao_4 = nlp("AROZ NOVAMARCA T 1 PREMI PCT 1KL") 
descricao_5 = nlp("ARROS PARBOILIZADO NOVMAR PREM T1 PACOTE 1 KL") 

# display
spacy.displacy.render(descricao_1, style="ent") 
spacy.displacy.render(descricao_2, style="ent") 
spacy.displacy.render(descricao_3, style="ent") 
spacy.displacy.render(descricao_4, style="ent") 
spacy.displacy.render(descricao_5, style="ent") 

In [71]:
# construção da descrição normalizda a partir das descrições de estabelecimento já analisadas pelo modelo NER.
# é necessário ter importado o arquivo das descricoes normalizadas enriquecidas no dataframe descrições

res_dic = {} #inicializa dicionário com o resultado da análise conjunta de cada descrição de estabelecimento

desc=[descricao_1, descricao_2, descricao_3, descricao_4, descricao_5] #inicializa lista com descrições de estabelecimento

for doc in desc: # para cada decrição
    if doc.ents: # se existe pelo menos uma entidade nesta descrição
        for ent in doc.ents: # para cada entidade da descrição
            tipo_entidade = ent.label_ # rótulo da entidade, p.ex. PROD
            entidade=ent.text # texto relacionado ao rótulo
            # a partir da base normalizada enriquecida, extrair entidades únicas, sem NaN e limpando o índice
            lista_entidades=descricoes[tipo_entidade].drop_duplicates().dropna().reset_index(drop=True)
            dic_entidades={}
            for i in range(len(lista_entidades)):
                #calcula a similaridade
                dic_entidades.update({lista_entidades[i]: similaridade(lista_entidades[i],entidade)})
            #ordena da maior para menor similaridade
            dic_entidades=sorted(dic_entidades.items(), key=itemgetter(1), reverse=True)
            if tipo_entidade == 'MARCA': #cutoff do atributo MARCA é maior para evitar confundir com uma marca já existente
                cutoff = 0.8
            else:
                cutoff = 0.6     
            if dic_entidades[0][1] > cutoff: #deu match
                if tipo_entidade in res_dic: #escolhe a entidade com a mario similaridade - posição 0 da lista
                  res_dic[tipo_entidade].append(dic_entidades[0][0])
                else:
                  res_dic[tipo_entidade] = [dic_entidades[0][0]]
            else:#se não deu match então utilizar a própria descrição do estabelecimento
                if tipo_entidade in res_dic:
                  res_dic[tipo_entidade].append(entidade)
                else:
                  res_dic[tipo_entidade] = [entidade]

descricao_normalizada='' #inicializa a descrição normalizada

#construção da descrição normalizada 
for atributo in ['PROD', 'PROD_X', 'TIPO', 'MARCA', 'MARCA_X','EMBAL','PESO']:
    lista_entidades = res_dic[atributo] # inicializa a lista de entidades
    frequencias = collections.Counter(lista_entidades) # calcula a frequencia de cada elemento da lista de entidades
    max_freq = max(frequencias.values()) # elemento de máxima frequencia
    if max_freq != 1: # existe uma entidade com frequencia maior que as demais      
        desc_norm_i = max(lista_entidades, key=frequencias.get) # entidade associada a máxima frequencia
    else:
        desc_norm_i=max(list(frequencias.keys()), key=len)
    descricao_normalizada=descricao_normalizada+' '+desc_norm_i
    
print('--- Descrição Normalizada ---')
print(descricao_normalizada.strip())


--- Descrição Normalizada ---
ARROZ PARBOILIZADO TIPO 1 NOVAMARCA PREMIUM PACOTE 1KG


# Funções

In [7]:
# Função similaridade
# cálculo do grau de similaridade entre duas strings
#
# argumentos
# a: string
# b: string
#
# saída
# grau de similaridade que pode variar de 0 a 1
#
def similaridade(a, b):
    return min(SequenceMatcher(None, a, b).ratio(),SequenceMatcher(None, b, a).ratio())

In [ ]:
# Função mais_similar
# escolhe a palavra mais similar entre as palavras do texto; limite de similaridade de 0.5
#
# argumentos
# palavra: string que se quer relacionar com a palavra mais similar do texto
# texto: texto onde se quer procurar a palavra mais similar
#
# saída
# palavra mais similar, posicao inicial, posicao final, grau de similaridade
#
def mais_similar(palavra, texto):
    if not isinstance(palavra, str): #se a palavra não for uma string
        return '',0.0,0,0
    texto_arrumado=" ".join(texto.split()) #retira brancos desnecessários
    palavra_arrumada=" ".join(palavra.split()) #retira brancos desnecessários
    
    lista_similaridade=()
    n=len(texto_arrumado) #comprimento do texto_arrumado
    k=len(palavra_arrumada)#comprimento da palavra_arrumada
    
    for i in range(k,0,-1): #loop que percorre a palavra+1 até um mínimo de 2 caracteres
        for j in range(0,n-i): #loop que percorre o texto
            trecho=" ".join(texto_arrumado[j:j+i+1].split()) #slice do texto
            #lista com todas as variações
            lista_similaridade=lista_similaridade+( (trecho,similaridade(trecho,palavra_arrumada)), )
    
    #trecho do texto mais similar
    if lista_similaridade==():
        trecho=''
    else:
        trecho=tuple(sorted(lista_similaridade, key=lambda item: item[1], reverse=True))[0][0] 
    
    #calculo da posicao final
    pos_fin=texto_arrumado.find(" ",texto_arrumado.find(trecho)+len(trecho))
    if pos_fin == -1: pos_fin=len(texto_arrumado)

    #calculo da posicao inicial
    texto_arrumado_invertido=texto_arrumado[::-1] #inverte texto
    trecho_invertido=trecho[::-1] #inverte trecho    
    pos_ini=texto_arrumado_invertido.find(" ",texto_arrumado_invertido.find(trecho_invertido)+len(trecho))
    if pos_ini == -1: pos_ini=len(texto_arrumado_invertido)
    pos_ini = len(texto_arrumado_invertido)-pos_ini
    
    token=texto_arrumado[pos_ini:pos_fin] #trecho com maior similaridade
    
    #grau de similaridade
    grau_similaridade=similaridade(token,palavra) 
    
    if grau_similaridade<0.5: #limite de 0.5 para similaridade
        return '',0.0,0,0
    
    return token, grau_similaridade, pos_ini, pos_fin

In [ ]:
# Função overlap
# verifica overlap entre duas faixas de posição (start1, end1) e (start1, end1)
#
# argumentos
# s1: start da faixa 1
# e1: end da faixa 1
# s2: start da faixa 2
# e2: end da faixa 2
#
# saída
# True ou False para overlap
#
def overlap(s1,e1,s2,e2):
    return (((s2>=s1)and(s2<e1))or((e2>s1)and(e2<=e1))or((s1>=s2)and(s1<e2))or((e1>s2)and(e1<=e2)))and not((s1==e1)or(s2==e2))

In [ ]:
# Função elimina_conflitos
# elimina eventuais conflitos de overlap (mesma palavra para mais de um atributo)
#
# argumento
# analise: lista com a analise de similaridade
#
# saída
# lista com analise sem conflitos
#
def elimina_conflitos(analise):
    
    for i in range(len(analise)):
        for j in range(i,len(analise)):
            if i!=j:
                s1=analise[i][3]
                e1=analise[i][4]
                s2=analise[j][3]
                e2=analise[j][4]
                if overlap(s1,e1,s2,e2):
                    if analise[i][2]>=analise[j][2]: #i tem mais similaridade que j
                        analise[j][1]=''
                        analise[j][2]=0.0
                        analise[j][3]=0
                        analise[j][4]=0
                    else:
                        analise[i][1]=''
                        analise[i][2]=0.0
                        analise[i][3]=0
                        analise[i][4]=0   
      
    return analise

In [3]:
# Função mostra_ents
# mostra as entidades de uma descrição
#
# argumentos
# doc: string com a descrição do estabelecimento
#
# saída
# impressão das entidades da descrição
#
def mostra_ents(doc): 
    if doc.ents: 
        for ent in doc.ents: 
            print(ent.label_+' - '+ent.text) 
    else: 
        print('Nenhuma entidade encontrada.')